<a href="https://colab.research.google.com/github/Sakinat-Folorunso/OOU_CSC309_Artificial_Intelligence/blob/main/notebooks/CSC309_Week03_Search_BFS_DFS_Student_Centred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC309 – Artificial Intelligence  
**Week 3 Lab:** Search & Problem Solving — BFS/DFS on Classic Problems

**Instructor:** Dr Sakinat Folorunso

**Title:** Associate Professor of AI Systems and FAIR Data **Department:** Computer Sciences, Olabisi Onabanjo University, Ago-Iwoye, Ogun State, Nigeria

**Course Code:** CSC 309

**Mode:** Student‑centred, hands‑on in Google Colab

> Every code cell is commented line‑by‑line so you can follow the logic precisely.

## How to use this notebook
1. Start with the **Group Log** and **Do Now**.  
2. Run the **Setup** cell once.  
3. Work through **Tasks**. Edit only cells marked **`# TODO(Student)`**.  
4. Use **Quick Checks** to test your understanding.  
5. Finish with the **Reflection**. If you finish early, try the **Extensions**.

In [ ]:
#@title 🧑🏽‍🤝‍🧑🏾 Group Log (fill before you start)
# The '#@param' annotations create form fields in Colab for easy input.

group_members = "Type names here"  #@param {type:"string"}  # Names of teammates
roles_notes = "Driver/Navigator, decisions, questions"  #@param {type:"string"}  # Short working notes

print("👥 Group:", group_members)        # Echo the group list for confirmation
print("📝 Notes:", roles_notes)          # Echo the notes so they're preserved in output

### Learning Objectives
- Formulate problems as **state spaces**.  
- Implement **BFS** and **DFS** with path reconstruction.  
- Discuss completeness/optimality and **combinatorial explosion**.

In [ ]:
#@title 🔧 Setup (no extra installs)
# BFS/DFS rely only on Python's standard library.

from collections import deque
import heapq     # 'deque' gives an efficient queue for BFS
print("✅ Setup complete for Week 3.")

In [ ]:
# -------------------------------------------------------------------------
# 1. Search algorithms (with expansion counters)
# -------------------------------------------------------------------------

def bfs(start, is_goal, neighbors):
    """Classic BFS – returns (path, nodes_expanded)"""
    frontier = deque([start])
    parent   = {start: None}
    expanded = 0

    while frontier:
        s = frontier.popleft()
        expanded += 1
        if is_goal(s):
            path = []
            while s is not None:
                path.append(s)
                s = parent[s]
            return list(reversed(path)), expanded

        for n in neighbors(s):
            if n not in parent:
                parent[n] = s
                frontier.append(n)
    return None, expanded

In [ ]:
def dfs(start, is_goal, neighbors, limit=20000):
    """DFS with expansion counter and safety limit"""
    stack    = [start]
    parent   = {start: None}
    expanded = 0

    while stack and expanded < limit:
        s = stack.pop()
        expanded += 1
        if is_goal(s):
            path = []
            while s is not None:
                path.append(s)
                s = parent[s]
            return list(reversed(path)), expanded

        for n in neighbors(s):
            if n not in parent:
                parent[n] = s
                stack.append(n)
    return None, expanded


def ucs(start, is_goal, neighbors, cost=lambda u, v: 1):
    """Uniform-Cost Search – works with any edge costs"""
    frontier = []                                   # priority queue
    heapq.heappush(frontier, (0, start))           # (g, state)
    came_from = {start: None}
    g_score   = {start: 0}
    expanded  = 0

    while frontier:
        current_g, current = heapq.heappop(frontier)
        expanded += 1

        if is_goal(current):
            path = []
            while current is not None:
                path.append(current)
                current = came_from[current]
            return list(reversed(path)), expanded

        for neigh in neighbors(current):
            edge_cost = cost(current, neigh)
            new_g = g_score[current] + edge_cost

            if neigh not in g_score or new_g < g_score[neigh]:
                came_from[neigh] = current
                g_score[neigh]   = new_g
                heapq.heappush(frontier, (new_g, neigh))

    return None, expanded


# -------------------------------------------------------------------------
# 2. Missionaries & Cannibals problem definition
# -------------------------------------------------------------------------

def mc_neighbors(state):
    """Legal moves for the classic 3-missionaries / 3-cannibals puzzle"""
    M, C, boat = state                     # boat: 0 = left, 1 = right
    possible_moves = [(1,0), (2,0), (0,1), (0,2), (1,1)]

    successors = []
    for m, c in possible_moves:
        if boat == 0:                      # boat on left → move to right
            new_M, new_C, new_boat = M - m, C - c, 1
        else:                              # boat on right → move to left
            new_M, new_C, new_boat = M + m, C + c, 0

        if not (0 <= new_M <= 3 and 0 <= new_C <= 3):
            continue

        # safety checks
        left_safe  = (new_M == 0 or new_M >= new_C)
        right_M    = 3 - new_M
        right_C    = 3 - new_C
        right_safe = (right_M == 0 or right_M >= right_C)

        if left_safe and right_safe:
            successors.append((new_M, new_C, new_boat))

    return successors


# -------------------------------------------------------------------------
# 3. Run the three algorithms
# -------------------------------------------------------------------------

start = (3, 3, 0)      # 3M 3C boat on left
goal  = (0, 0, 1)      # everything on right

path_bfs, exp_bfs = bfs(start, lambda s: s == goal, mc_neighbors)
path_dfs, exp_dfs = dfs(start, lambda s: s == goal, mc_neighbors)
path_ucs, exp_ucs = ucs(start, lambda s: s == goal, mc_neighbors)

# -------------------------------------------------------------------------
# 4. Results
# -------------------------------------------------------------------------

print("Missionaries & Cannibals – Search comparison")
print("=" * 60)
print(f"{'Algorithm':<8} {'Solution':<9} {'Steps':<6} {'Nodes expanded'}")
print("-" * 60)
print(f"{'BFS':<8} {'Yes' if path_bfs else 'No':<9} {len(path_bfs) if path_bfs else '-':<6} {exp_bfs}")
print(f"{'DFS':<8} {'Yes' if path_dfs else 'No':<9} {len(path_dfs) if path_dfs else '-':<6} {exp_dfs}")
print(f"{'UCS':<8} {'Yes' if path_ucs else 'No':<9}")

### Task — Uniform‑Cost Search (UCS)
Extend BFS into **UCS** using a priority queue; assign unit cost to each boat move.  
Then compare DFS/BFS/UCS by **number of states expanded**.